# **Homework - Session 3**

## **Import libs**

In [1]:
import numpy as np
import pandas as pd

## **Load Data**

In [191]:
data = pd.read_csv("https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv")

In [192]:
data.shape

(48895, 16)

In [4]:
data.head(2)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355


## **Usefull Data**

In [194]:
df = data[[

    'neighbourhood_group',
    'room_type',
    'latitude',
    'longitude',
    'price',
    'minimum_nights',
    'number_of_reviews',
    'reviews_per_month',
    'calculated_host_listings_count',
    'availability_365'
]]

In [195]:
df.shape

(48895, 10)

## **Question 1**
What is the most frequent observation (mode) for the column _**'neighbourhood_group'**_?

In [196]:
df.neighbourhood_group.value_counts()

Manhattan        21661
Brooklyn         20104
Queens            5666
Bronx             1091
Staten Island      373
Name: neighbourhood_group, dtype: int64

##  **Split the data**
Split your data in train/val/test sets, with 60%/20%/20% distribution.
Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
Make sure that the target value ('price') is not in your dataframe.

In [17]:
from sklearn.model_selection import train_test_split

In [198]:
df.isnull().sum()

neighbourhood_group                   0
room_type                             0
latitude                              0
longitude                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [199]:
df.fillna(0, inplace=True)

/home/vscode/.local/lib/python3.7/site-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [201]:
df.isnull().sum()

neighbourhood_group               0
room_type                         0
latitude                          0
longitude                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

In [95]:
target = df.price
features = df.drop("price", axis=1)

In [96]:
# np.random.seed(42)

X_full_train, X_test, y_full_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_full_train, y_full_train, test_size=0.25, random_state=42)

print("train set : ", X_train.shape, y_train.shape, round(len(X_train)/len(df),2))
print("val set : ", X_val.shape, y_val.shape, round(len(X_val)/len(df),2))
print("test set : ", X_test.shape, y_test.shape, round(len(X_test)/len(df),2))

train set :  (29337, 9) (29337,) 0.6
val set :  (9779, 9) (9779,) 0.2
test set :  (9779, 9) (9779,) 0.2


In [97]:
# Make sure that the target value ('price') is not in your dataframe
print('price' in X_train.columns)
print('price' in X_val.columns)
print('price' in X_test.columns)

False
False
False


## **Question 2**
Create the correlation matrix for the numerical features of your train dataset.  
In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.  
What are the two features that have the biggest correlation in this dataset?

In [98]:
X_train.select_dtypes(include=["int", "float"]).corr().unstack().sort_values(ascending=False)

latitude                        latitude                          1.000000
longitude                       longitude                         1.000000
calculated_host_listings_count  calculated_host_listings_count    1.000000
reviews_per_month               reviews_per_month                 1.000000
minimum_nights                  minimum_nights                    1.000000
number_of_reviews               number_of_reviews                 1.000000
availability_365                availability_365                  1.000000
number_of_reviews               reviews_per_month                 0.590374
reviews_per_month               number_of_reviews                 0.590374
availability_365                calculated_host_listings_count    0.225913
calculated_host_listings_count  availability_365                  0.225913
availability_365                number_of_reviews                 0.174477
number_of_reviews               availability_365                  0.174477
availability_365         

## **Make price binary**
We need to turn the price variable from numeric into binary.  
Let's create a variable _**above_average**_ which is **_1_** if the price is above (or equal to) **_152_**.

In [99]:
above_average = y_train.apply(lambda x : 1 if x >= 152 else 0)

In [100]:
above_average.shape

(29337,)

In [203]:
above_average.value_counts(normalize=True)

0    0.695204
1    0.304796
Name: price, dtype: float64

In [102]:
type(above_average)

pandas.core.series.Series

In [144]:
df['above_average'] = (df['price'] >= 152).values.astype(int)

/home/vscode/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## **Question 3**
Calculate the mutual information score with the (binarized) price for the two categorical variables that we have.   
Use the training set only.  
Which of these two variables has bigger score?  
Round it to 2 decimal digits using round(score, 2)

In [103]:
X_train.select_dtypes(include="object").columns

Index(['neighbourhood_group', 'room_type'], dtype='object')

In [104]:
from sklearn.metrics import mutual_info_score

In [105]:
round(mutual_info_score(above_average, X_train.neighbourhood_group),2)

0.05

In [106]:
round(mutual_info_score(above_average, X_train.room_type),2)

0.14

## **Question 4**
- Now let's train a logistic regression
- Remember that we have two categorical variables in the data. Include them using one-hot encoding.
- Fit the model on the training dataset.
    * To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    * model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
- Calculate the accuracy on the validation dataset and rount it to 2 decimal digits.

In [167]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [135]:
numerical = list(X_train.select_dtypes(include=["int", "float"]))
numerical

['latitude',
 'longitude',
 'minimum_nights',
 'number_of_reviews',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365']

In [136]:
categorical = list(X_train.select_dtypes(include="object"))
categorical

['neighbourhood_group', 'room_type']

In [146]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.2, random_state=42)
len(df_train), len(df_val), len(df_test)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [147]:
y_train = (df_train['price'] >= 152).values.astype(int)
y_val = (df_val['price'] >= 152).values.astype(int)
y_test = (df_test['price'] >= 152).values.astype(int)

del df_train['price']
del df_val['price']
del df_test['price']

In [150]:
features.columns

Index(['neighbourhood_group', 'room_type', 'latitude', 'longitude',
       'minimum_nights', 'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365'],
      dtype='object')

In [151]:
# one-hot encoding datasets
dv = DictVectorizer(sparse=False)

train_dict = df_train[features.columns].to_dict(orient='records')
val_dict = df_val[features.columns].to_dict(orient='records')

X_train = dv.fit_transform(train_dict)
X_val = dv.transform(val_dict)

In [168]:
# fit the model
model = LogisticRegression(solver='liblinear', C=1.0, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(random_state=42, solver='liblinear')

In [161]:
# use the model
y_pred = model.predict_proba(X_val)[:,1]

In [162]:
# accuracy with thresholg = 0.5
result_accuracy = (y_val == (y_pred >= 0.5)).mean()
round(result_accuracy,2)

0.79

## **Question 5**
- We have 9 features: 7 numerical features and 2 categorical.
- Let's find the least useful one using the feature elimination technique.
- Train a model with all these features (using the same parameters as in Q4).
- Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
- For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
- Which of following feature has the smallest difference?
    * neighbourhood_group
    * room_type
    * number_of_reviews
    * reviews_per_month
> note: the difference doesn't have to be positive

In [158]:
numerical

['latitude',
 'longitude',
 'minimum_nights',
 'number_of_reviews',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365']

In [159]:
categorical

['neighbourhood_group', 'room_type']

In [176]:
def func_accuracy(features):
    dv = DictVectorizer(sparse=False)
    train_dict = df_train[features].to_dict(orient='records')
    val_dict = df_val[features].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)
    X_val = dv.transform(val_dict)
    model = LogisticRegression(solver='liblinear', C=1.0, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)[:,1]

    # accuracy with threshold = 0.5
    acc = (y_val == (y_pred >= 0.5)).mean()
    return acc

In [171]:
all_features = numerical + categorical
all_features

['latitude',
 'longitude',
 'minimum_nights',
 'number_of_reviews',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365',
 'neighbourhood_group',
 'room_type']

In [180]:
all_features = numerical + categorical
diff = {}
for i in all_features:
    features_to_use = all_features.copy()
    features_to_use.remove(i)
    # print(features_to_use)
    # break
    acc = func_accuracy(features_to_use)
    diff[f"Diff. in accuracy without {i}"] = result_accuracy - acc

# sort diff
dict(sorted(diff.items(), key=lambda item: item[1]))

{'Diff. in accuracy without number_of_reviews': -0.0003834355828220559,
 'Diff. in accuracy without reviews_per_month': -0.0001278118609406853,
 'Diff. in accuracy without minimum_nights': 0.0003834355828220559,
 'Diff. in accuracy without calculated_host_listings_count': 0.0003834355828220559,
 'Diff. in accuracy without latitude': 0.005112474437627745,
 'Diff. in accuracy without longitude': 0.005240286298568542,
 'Diff. in accuracy without availability_365': 0.01111963190184051,
 'Diff. in accuracy without neighbourhood_group': 0.04115541922290389,
 'Diff. in accuracy without room_type': 0.06505623721881393}

In [ ]:
# smallest = 'number_of_reviews'

## **Question 6**
- For this question, we'll see how to use a linear regression model from Scikit-Learn
- We'll need to use the original column 'price'. Apply the logarithmic transformation to this column.
- Fit the Ridge regression model on the training data.
- This model has a parameter alpha. Let's try the following values: [0, 0.01, 0.1, 1, 10]
- Which of these alphas leads to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits.
If there are multiple options, select the smallest alpha.

In [ ]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.2, random_state=42)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

categorical = [col for col in df.columns if df[col].dtype == 'object']
numerical = [col for col in df.columns if col != "object"]

# log transformation
y_train = np.log1p(df_train['price'].values)
y_val = np.log1p(df_val['price'].values)
y_test = np.log1p(df_test['price'].values)

# delete target value
del df_train['price']
del df_val['price']
del df_test['price']

# cat. variables
dv = DictVectorizer(sparse=False)
train_dict = df_train[features].to_dict(orient='records')
val_dict = df_val[features].to_dict(orient='records')

X_train = dv.fit_transform(train_dict)
X_val = dv.transform(val_dict)

In [182]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [190]:
rmse_scores = {}
for alpha in [0, 0.01, 0.1, 1, 10]:
    model = Ridge(alpha=alpha)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse_score = np.sqrt(mean_squared_error(y_val, y_pred))
    # print(rmse_score)
    rmse_scores[f"alpha = {alpha} => rmse"] = rmse_score.round(3)
 
# rmse_scores
dict(sorted(rmse_scores.items(), key=lambda item: item[1]))

{'alpha = 0 => rmse': 0.376,
 'alpha = 0.01 => rmse': 0.376,
 'alpha = 0.1 => rmse': 0.376,
 'alpha = 1 => rmse': 0.376,
 'alpha = 10 => rmse': 0.376}

In [ ]:
# smallest alpha = 0